# difFUBAR

difFUBAR is an approximate Bayesian analysis of adaptive evolution. For each codon site, difFUBAR compares $\omega$, the non-synonymous to synonymous substitution rate ratio ($\beta/\alpha$, or $dN/dS$) between two predefined clades on a phylogeny.

## You need:

- A **codon** multiple sequence alignment, in `.fasta` format.
- A [Newick](https://en.wikipedia.org/wiki/Newick_format) phylogeny (typically estimated from the alignment) where the taxon names match the alignment, and where nodes have been tagged (eg. `taxon_name{G1}` in the phylogeny string).
 - There must be two different tags, which define the groups to be compared, and any untagged branches will not participate in the comparison.

## Tips:
- You can upload your tree to our [phylogeny tagging utility](https://murrellgroup.github.io/WebWidgets/phylotagger.html) to label clades.
- Try and avoid potentially problematic characters (space, colon, semicolon, brackets, etc) in your sequence names (clean these up before you construct the phylogeny to make sure the names match).
- Visually inspect your alignments, both as nucleotide sequences and as their translations (we like [AliView](https://ormbunkar.se/aliview/)).
- Package installation and precompilation takes some time on Colab.
 - **If you instead install and run Julia locally, you only ever need to do this once!**
 - Go [here](https://julialang.org/install/) for instructions on how to install Julia


This notebook will run with demo files. If you wish to analyze your own files, upload them using the Colab files menu on the left: image.png
→
image.png

In [1]:
#Install and load packages: (this can take some time the first time on Colab)
using Pkg
Pkg.add("Suppressor") #Suppressor, with @suppress and @suppress_err macros are just to prevent long outputs from clogging up the notebook example - you don't really need them.
using Suppressor
@suppress Pkg.rm(["Lux","Reactant"])
@suppress Pkg.add(["MolecularEvolution", "FASTX"])
@suppress Pkg.add(url = "https://github.com/MurrellGroup/CodonMolecularEvolution.jl")
using MolecularEvolution, FASTX, CodonMolecularEvolution

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed Suppressor ─ v0.2.8
    Updating `~/.julia/environments/v1.10/Project.toml`
  [fd094767] + Suppressor v0.2.8
    Updating `~/.julia/environments/v1.10/Manifest.toml`
  [fd094767] + Suppressor v0.2.8
Precompiling packages...
   3119.1 ms  ✓ Suppressor
  1 dependency successfully precompiled in 11 seconds. 459 already precompiled.


In [2]:
#Fetch the demo multiple sequence alignment, and tagged phylogeny:
@suppress run(`wget https://raw.githubusercontent.com/MurrellGroup/CodonMolecularEvolution.jl/refs/heads/main/docs/src/H5_subsampled.fasta`);
@suppress run(`wget https://raw.githubusercontent.com/MurrellGroup/CodonMolecularEvolution.jl/refs/heads/main/docs/src/H5_subsampled_tagged.tre`);
("H5_subsampled.fasta" in readdir() && "H5_subsampled_tagged.tre" in readdir()) && println("Files fetched!")

Files fetched!


In [8]:
seqnames,seqs = read_fasta("H5_subsampled.fasta"); #<- Reads in fasta alignment
treestring = readlines(open("H5_subsampled_tagged.tre"))[1]; #<- Reads in phylogeny (as a string)
df,results = @suppress_err difFUBAR(seqnames, seqs, treestring, ["{G1}", "{G2}"], "difFUBAR_H5_subsampled"); #<- Runs difFUBAR (note: the ["{G1}", "{G2}"] must match whatever tags you have on your tree)

Step 1: Initialization. If exports = true, tree showing the assignment of branches to groups/colors will be exported to: difFUBAR_H5_subsampled_tagged_input_tree.svg.
Step 2: Optimizing global codon model parameters.
Optimized single α,β LL=-61147.43019454108 with α=2.4528778755382867 and β=0.42151196711816985.
Step 3: Calculating grid of 1728-by-570 conditional likelihood values (the slowest step). Currently on:
0.0% 29.0% 58.0% 87.0% 
Step 4: Running Gibbs sampler to infer site categories.
Step 5: Tabulating and plotting. Detected sites:
Site 10 - P(ω1 > ω2):0.0; P(ω2 > ω1):0.9865; P(ω1 > 1):0.0035; P(ω2 > 1):0.2895
Site 11 - P(ω1 > ω2):0.0; P(ω2 > ω1):0.9945; P(ω1 > 1):0.252; P(ω2 > 1):0.9975
Site 12 - P(ω1 > ω2):0.0015; P(ω2 > ω1):0.9715; P(ω1 > 1):0.002; P(ω2 > 1):0.201
Site 112 - P(ω1 > ω2):0.996; P(ω2 > ω1):0.0; P(ω1 > 1):0.0065; P(ω2 > 1):0.0
Site 128 - P(ω1 > ω2):0.9845; P(ω2 > ω1):0.0; P(ω1 > 1):0.022; P(ω2 > 1):0.0
Site 145 - P(ω1 > ω2):0.0; P(ω2 > ω1):0.99; P(ω1 > 1):0.0005

## !!! You can now download the analysis outputs, plots, etc, in the left panel: image.png !!!

In [11]:
|#Retabulate, plot, etc processed data with a different posterior threshold (we'll make this simpler):
@suppress_err difFUBAR_tabulate("difFUBAR_H5_subsampled_0.85", 0.85, results[1:4]...);

Step 5: Tabulating and plotting. Detected sites:
Site 10 - P(ω1 > ω2):0.0; P(ω2 > ω1):0.9865; P(ω1 > 1):0.0035; P(ω2 > 1):0.2895
Site 11 - P(ω1 > ω2):0.0; P(ω2 > ω1):0.9945; P(ω1 > 1):0.252; P(ω2 > 1):0.9975
Site 12 - P(ω1 > ω2):0.0015; P(ω2 > ω1):0.9715; P(ω1 > 1):0.002; P(ω2 > 1):0.201
Site 16 - P(ω1 > ω2):0.013; P(ω2 > ω1):0.906; P(ω1 > 1):0.006; P(ω2 > 1):0.13
Site 18 - P(ω1 > ω2):0.8875; P(ω2 > ω1):0.016; P(ω1 > 1):0.0945; P(ω2 > 1):0.0005
Site 32 - P(ω1 > ω2):0.921; P(ω2 > ω1):0.001; P(ω1 > 1):0.0055; P(ω2 > 1):0.0
Site 51 - P(ω1 > ω2):0.017; P(ω2 > ω1):0.8785; P(ω1 > 1):0.0; P(ω2 > 1):0.0
Site 69 - P(ω1 > ω2):0.0205; P(ω2 > ω1):0.872; P(ω1 > 1):0.0; P(ω2 > 1):0.0005
Site 87 - P(ω1 > ω2):0.009; P(ω2 > ω1):0.869; P(ω1 > 1):0.008; P(ω2 > 1):0.175
Site 89 - P(ω1 > ω2):0.8755; P(ω2 > ω1):0.004; P(ω1 > 1):0.0; P(ω2 > 1):0.0
Site 111 - P(ω1 > ω2):0.007; P(ω2 > ω1):0.9275; P(ω1 > 1):0.0; P(ω2 > 1):0.007
Site 112 - P(ω1 > ω2):0.996; P(ω2 > ω1):0.0; P(ω1 > 1):0.0065; P(ω2 > 1):0.0
Site 12

<embed src="difFUBAR_H5_subsampled_tagged_input_tree.svg" type="image/svg+xml" />